# NHS Bed Utilization ETL + Validation

In [ ]:
import pandas as pd
import os
from pathlib import Path

# Path to your local data folder
data_dir = Path("nhs_data")

# Recursively collect all CSV files
csv_files = list(data_dir.rglob("*.csv"))
print(f"Found {len(csv_files)} CSV files.")
csv_files[:5]  # show sample file paths


In [ ]:
# Load and concatenate all CSV files into one DataFrame
dfs = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        df['source_file'] = file.name
        df['year'] = file.parts[-2]
        df['quarter'] = file.stem.split('_')[-1]
        dfs.append(df)
    except Exception as e:
        print(f"Failed to read {file}: {e}")

raw_df = pd.concat(dfs, ignore_index=True)
print(f"Combined dataset shape: {raw_df.shape}")
raw_df.head()


In [ ]:
# Standardize column names (example, adjust to your actual column names)
raw_df.columns = raw_df.columns.str.strip().str.lower().str.replace(" ", "_")

# Rename important fields if needed
# Example: raw_df.rename(columns={"occupied_beds": "occupied", "available_beds": "available"}, inplace=True)


In [ ]:
# Feature engineering: Add period and occupancy rate
raw_df['period'] = raw_df['year'] + "_" + raw_df['quarter']
raw_df['occupancy_rate'] = raw_df['occupied_bed_days'] / raw_df['available_bed_days']


In [ ]:
# Basic validation
print("Missing values summary:")
print(raw_df.isnull().sum())

print("\nInvalid occupancy rates:")
print(raw_df[(raw_df['occupancy_rate'] > 1.0) | (raw_df['occupancy_rate'] < 0)])

print("\nDuplicates:")
print(raw_df.duplicated().sum())

print("\nValue ranges:")
print(raw_df[['available_bed_days', 'occupied_bed_days', 'occupancy_rate']].describe())


In [ ]:
# Export cleaned data
output_path = Path("processed")
output_path.mkdir(exist_ok=True)
raw_df.to_csv(output_path / "cleaned_nhs_bed_data.csv", index=False)
print("✅ Cleaned data saved to: processed/cleaned_nhs_bed_data.csv")
